In [64]:
import tensorflow as tf
from functools import partial
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler

# 1. batch normalization

In [3]:
n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
graph=tf.Graph()
with graph.as_default():
    with tf.variable_scope("inputs", reuse=tf.AUTO_REUSE):
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
        y = tf.placeholder(tf.int64, shape=(None), name='y')

In [4]:
# build the network
with graph.as_default():
    with tf.variable_scope("inputs", reuse=tf.AUTO_REUSE):
        with tf.name_scope("dnn"):
            training = tf.placeholder_with_default(False, shape=(), name='training')
            my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)
            hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
            bn1 = my_batch_norm_layer(hidden1)
            bn1_act = tf.nn.elu(bn1)
            hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
            bn2 = my_batch_norm_layer(hidden2)
            bn2_act = tf.nn.elu(bn1)
            logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
            logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)

In [5]:
with graph.as_default():
    with tf.name_scope("loss"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

In [6]:
# optimizer
with graph.as_default():
    learning_rate = 0.01
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)
        # gradient clip
#         threshold = 1.0
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#         grads_and_vars = optimizer.compute_gradients(loss)
#         capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
#         training_op = optimizer.apply_gradients(capped_gvs)

In [7]:
# define evaluation
with graph.as_default():
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [8]:
with graph.as_default():
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [ ]:
# download data
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/")

In [ ]:
def normalize(images):
    standard_scaler = StandardScaler()
    res = standard_scaler.fit_transform(images)
    return res

In [38]:
mnist = pd.read_csv("/Users/lgrcyanny/Codecookies/machine-learning-workspace/datasets/mnist/mnist_train.csv", header=None)
mnist_test = pd.read_csv("/Users/lgrcyanny/Codecookies/machine-learning-workspace/datasets/mnist/mnist_test.csv", header=None)

In [69]:
mnist_test_images = mnist_test.loc[:, 1:]
mnist_test_labels = mnist_test.loc[:, 0]
mnist_test_images = normalize(mnist_test_images)

In [70]:
def next_batch(batch_size=50):
    batch = mnist.sample(batch_size)
    x_batch = batch.loc[:, 1:]
    x_batch = normalize(x_batch)
    y_batch = batch.loc[:, 0]
    return x_batch, y_batch

In [43]:
n_epochs = 40
batch_size = 50
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
num_examples, num_columns = mnist.shape
with tf.Session(graph=graph) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(num_examples // batch_size):
#             x_batch, y_batch = mnist.train.next_batch(batch_size)
            x_batch, y_batch = next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={X: x_batch, y: y_batch, training: True})
        acc_train = sess.run(accuracy, feed_dict={X: x_batch, y: y_batch, training: True})
        acc_test = sess.run(accuracy, feed_dict={X: mnist_test_images, y: mnist_test_labels, training: True})
        print("epoch: {}, accuracy_train: {}, accuracy_test: {}".format(epoch, acc_train, acc_test))
    save_path = saver.save(sess, "/tmp/tensorflow/my_model_final.ckpt")

epoch: 0, accuracy_train: 0.920000016689, accuracy_test: 0.913900017738
epoch: 1, accuracy_train: 0.939999997616, accuracy_test: 0.936699986458
epoch: 2, accuracy_train: 0.939999997616, accuracy_test: 0.94730001688
epoch: 3, accuracy_train: 0.980000019073, accuracy_test: 0.95609998703
epoch: 4, accuracy_train: 0.959999978542, accuracy_test: 0.960799992085
epoch: 5, accuracy_train: 0.959999978542, accuracy_test: 0.963599979877
epoch: 6, accuracy_train: 0.980000019073, accuracy_test: 0.968299984932
epoch: 7, accuracy_train: 1.0, accuracy_test: 0.968299984932
epoch: 8, accuracy_train: 1.0, accuracy_test: 0.970499992371
epoch: 9, accuracy_train: 1.0, accuracy_test: 0.97159999609
epoch: 10, accuracy_train: 0.980000019073, accuracy_test: 0.973500013351
epoch: 11, accuracy_train: 1.0, accuracy_test: 0.974600017071
epoch: 12, accuracy_train: 1.0, accuracy_test: 0.973900020123
epoch: 13, accuracy_train: 1.0, accuracy_test: 0.97469997406
epoch: 14, accuracy_train: 1.0, accuracy_test: 0.975199997

# 2. Reuse layers 

In [11]:
![image.png](attachment:image.png)

/bin/sh: -c: line 0: syntax error near unexpected token `attachment:image.png'
/bin/sh: -c: line 0: `[image.png](attachment:image.png)'


In [ ]:
reuse_saver = tf.train.import_meta_graph("/tmp/tensorflow/my_model_final.ckpt.meta")

In [ ]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [ ]:
X = tf.get_default_graph().get_tensor_by_name("inputs/X:0")
y = tf.get_default_graph().get_tensor_by_name("inputs/y:0")
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("train/GradientDescent")

In [ ]:
with tf.Session() as sess:
     reuse_saver.restore(sess, "/tmp/tensorflow/my_model_final.ckpt")

# 3. Faster Optimizer

In [ ]:
with graph.as_default():
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)

# 4. Regularization

In [85]:
dropout_rate = 0.05  # == 1 - keep_prob
graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope("inputs", reuse=tf.AUTO_REUSE):
        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
        y = tf.placeholder(tf.int64, shape=(None), name='y')
        
with graph.as_default():
    with tf.name_scope("dnn_with_dropout"):
        with tf.variable_scope("dnn", reuse=tf.AUTO_REUSE):
            training = tf.placeholder_with_default(False, shape=(), name='training')
            he_init = tf.contrib.layers.variance_scaling_initializer()
            X_drop = tf.layers.dropout(X, dropout_rate, training=training)
            hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                                      name="hidden1",  kernel_initializer=he_init)
            hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
            hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,
                                      name="hidden2")
            hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
            logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

In [86]:
with graph.as_default():
    with tf.name_scope("dropout_loss"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

In [87]:
# with graph.as_default():
#     initial_learning_rate = 0.1
#     decay_steps = 10000
#     decay_rate = 1/10
#     global_step = tf.Variable(0, trainable=False, name="global_step")
#     learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
#                                                decay_steps, decay_rate)
#     optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9, use_nesterov=True)
#     training_op = optimizer.minimize(loss, global_step=global_step)
with graph.as_default():
    learning_rate = 0.01
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)

In [88]:
with graph.as_default():
    init = tf.global_variables_initializer()

In [89]:
with graph.as_default():
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [90]:
n_epochs = 40
batch_size = 50
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
num_examples, num_columns = mnist.shape
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(num_examples // batch_size):
            x_batch, y_batch = next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={X: x_batch, y: y_batch, training: True})
        acc_train = sess.run(accuracy, feed_dict={X: x_batch, y: y_batch, training: True})
        acc_test = sess.run(accuracy, feed_dict={X: mnist_test_images, y: mnist_test_labels, training: False})
        print("epoch: {}, accuracy_train: {}, accuracy_test: {}".format(epoch, acc_train, acc_test))

epoch: 0, accuracy_train: 0.939999997616, accuracy_test: 0.919700026512
epoch: 1, accuracy_train: 0.860000014305, accuracy_test: 0.937200009823
epoch: 2, accuracy_train: 0.899999976158, accuracy_test: 0.944400012493
epoch: 3, accuracy_train: 0.939999997616, accuracy_test: 0.949100017548
epoch: 4, accuracy_train: 0.939999997616, accuracy_test: 0.954800009727
epoch: 5, accuracy_train: 0.920000016689, accuracy_test: 0.957499980927
epoch: 6, accuracy_train: 0.939999997616, accuracy_test: 0.95870000124
epoch: 7, accuracy_train: 0.939999997616, accuracy_test: 0.962300002575
epoch: 8, accuracy_train: 0.980000019073, accuracy_test: 0.963599979877
epoch: 9, accuracy_train: 0.980000019073, accuracy_test: 0.966199994087
epoch: 10, accuracy_train: 0.980000019073, accuracy_test: 0.966099977493
epoch: 11, accuracy_train: 1.0, accuracy_test: 0.96590000391
epoch: 12, accuracy_train: 0.980000019073, accuracy_test: 0.967100024223
epoch: 13, accuracy_train: 1.0, accuracy_test: 0.968100011349
epoch: 14, a